## Обязательные требования

- Github-репозитории управляются из среды разработки. Коммиты созданные через сайт не засчитываются.
- Выполненная работа должна содержать несколько коммитов, отражающих процесс выполнения вами лабораторной работы. Один коммит будет приравниваться к списанной работе.
- Сообщения коммитов должны исчерпывающе описывать изменения, которые они содержат.
- Необходимо разделить код на функции так, что каждая функция имеет небольшую зону ответсвенности и решает одну изолированную задачу, которую не повторяет ни одна другая функция.
- Необходимо оформить ноутбук для удобства сдачи лабораторной работы.
- Код оформить в соответствии с PEP8.
- Функции должны иметь dosctring.
- Сигнатура функции оформлена с использованием type-hinting.

Для создания и обучения моделей необходимо использовать фреймворк `PyTorch`

Использование других фреймворков глубокого обучения (TensorFlow, Keras, Caffe и т.п.) в рамках данной лабораторной работы не допускается 😢


## Задание


1.   Необходимо загрузить исходный набор данных и соответствующие метки классов.
2.   Произвести разделение загруженного набора данных на обучающую, тестовую и валидационую выборки (в соотношении 80:10:10). Проверить, что сформированные выборки сбалансированы.
3.   Написать модель нейронной сети для решения задачи классификации.
4.   Описать пайплайн предобработки данных. **ВАЖНО**: что так как ваш вариант предполагает работу с текстом, то необходимо выполниить векторизацию данных (подробности в туториале).
4.   Написать `train loop` (цикл обучения). Провести эксперименты по обучению с различными значениями параметров `learning rate` (скорость обучения) и `batch size` (размер мини-пакета). Выбрать по 3 значения для `learning rate` и `batch size` (итоговое количество экспериментов будет 9).
5.   Для каждого проведенного эксперимента вывести графики для значения функции потерь (ось `x` - итерация обучения/номер эпохи; ось `y` - значение функции потерь) и выбранной метрики качества (ось `x` - итерация обучения/номер эпохи; ось `y` - значение метрики качества). Графики необходимо выводить как для обучающей, так и для валидационной выборки.
6.   Оценить качество работы модели на тестовой выборке.
7.   Сделайте выводы по полученным результатам проведенных экспериментов. Какую модель из всех полученных стоит использовать?
8.   Сохранить обученную модель.
9.   Выполните повторную инициализацию модели и загрузку весов.  Продемонстрируйте работоспособность модели (пропустите через нее какой-то отзыв/рецензию и выведите результат).



## Снова спарсим наш любимый сайт

``

In [1]:
import os
import re
import shutil
import pandas as pd

from modules.Analysis_Data__appProgram.modules import annotation as ann

In [2]:
ann_base = "ann_base"


current_path = os.getcwd()
parrent_path = os.path.join(current_path, "..")

if not os.path.exists(ann_base):
    os.mkdir(ann_base)

else:
    shutil.rmtree(ann_base)
    os.mkdir(ann_base)


dataset_path = os.path.join("datasets", "dataset")
ann_path = os.path.join(ann_base, "ann.csv")

df = ann.annotation_dataset(dataset_path, ann_path)
df.drop("rel_path", axis=1, inplace=True)

df.rename(columns={"class": "stars"}, inplace=True)

In [3]:
def read_file(path: str) -> str:
    """
    Функция для чтения файла и перевода его в строку
    Parameters
    ----------
    path: str
      Путь к файлу

    Returns
    -------
    str
    Строка, полученная из даннного файла
    """

    with open(path, "r", encoding="utf-8") as file:
        return file.read()


df["text"] = df["abs_path"].apply(read_file)

df.dropna(subset=['text'], inplace=True)
df.drop_duplicates(subset=["text"], inplace=True)

In [4]:
def count_words(string: str) -> int:
    """
    Возвращает количество слов в строке
    Parameters
    ----------
    string: str
      Исходная строка

    Returns
    -------
    int
    Количество слов в строке
    """

    cleaned_string = re.sub(r"[^\w\s]", " ", string)

    words = cleaned_string.split()

    return len(words)


df["count_words"] = df["text"].apply(count_words)
df["stars"] = df["stars"].astype(int) - 1
df


,abs_path,stars,text,count_words
0,/home/nick/Desktop/Univ/Programming/end_5_lab/...,0,Книга полна разочарований.Автор позиционирует ...,48
1,/home/nick/Desktop/Univ/Programming/end_5_lab/...,0,Когда я повернула не туда? Почему никто мне не...,118
2,/home/nick/Desktop/Univ/Programming/end_5_lab/...,0,"Претенциозное ничто, как и предыдущая книга в ...",22
4,/home/nick/Desktop/Univ/Programming/end_5_lab/...,0,Вроде бы нельзя воспринимать эту книгу букваль...,30
5,/home/nick/Desktop/Univ/Programming/end_5_lab/...,0,Посыл у книги важный и нужный. Отчетливо понят...,126
...,...,...,...,...
1500,/home/nick/Desktop/Univ/Programming/end_5_lab/...,4,Книга 100% заслуживает вашего внимания и време...,119
1501,/home/nick/Desktop/Univ/Programming/end_5_lab/...,4,"Казалось бы, в книге изложены те истины, котор...",40
1502,/home/nick/Desktop/Univ/Programming/end_5_lab/...,4,Автор не разочаровывает к моему большому удово...,33
1503,/home/nick/Desktop/Univ/Programming/end_5_lab/...,4,"красивый и богатый язык, тонкий юмор, классные...",126


In [ ]:
# df.to_csv("sdk.csv")

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import os

import nltk
from nltk.tokenize import word_tokenize
from nltk import PorterStemmer
from nltk import WordNetLemmatizer
from nltk.stem import SnowballStemmer
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords

nltk.download("punkt")
nltk.download("stopwords")


[nltk_data] Downloading package punkt to /home/nick/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/nick/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
remove_non_alphabets =lambda x: re.sub(r'[^а-яА-Я]',' ',x)

tokenize = lambda x: word_tokenize(x)

ps = SnowballStemmer("russian")
stem = lambda w: [ ps.stem(x) for x in w ]

morph = MorphAnalyzer()
leammtizer = lambda w: [ morph.parse(x)[0].normal_form for x in w ]

stop_words = set(stopwords.words("russian"))

In [7]:
strka = """99 С другой стороны дальнейшее развитие различных форм деятельности требует от нас системного анализа новых предложений!

Соображения высшего порядка, а также начало повседневной работы по формированию позиции представляет собой интересный эксперимент проверк
и дальнейших направлений развития проекта! Повседневная практика показывает, что дальнейшее развитие различных форм деятельности играет важную роль в формировании новых предложений?

Практический опыт показывает, что новая модель организационной деятельности играет важную роль в формировании направлений 
прогрессивного развития! Дорогие друзья, начало повседневной работы по формированию позиции позволяет выполнить важнейшие задания по 
разработке системы обучения кадров, соответствующей насущным потребностям.

Повседневная практика показывает, что реализация намеченного плана развития представляет собой интересный эксперимент проверки..."""

strka = remove_non_alphabets(strka)
strka

strka = tokenize(strka)
strka

strka = stem(strka)
strka

leammtizer(strka)
strka


['с',
 'друг',
 'сторон',
 'дальн',
 'развит',
 'различн',
 'форм',
 'деятельн',
 'треб',
 'от',
 'нас',
 'системн',
 'анализ',
 'нов',
 'предложен',
 'соображен',
 'высш',
 'порядк',
 'а',
 'такж',
 'нача',
 'повседневн',
 'работ',
 'по',
 'формирован',
 'позиц',
 'представля',
 'соб',
 'интересн',
 'эксперимент',
 'проверк',
 'и',
 'дальн',
 'направлен',
 'развит',
 'проект',
 'повседневн',
 'практик',
 'показыва',
 'что',
 'дальн',
 'развит',
 'различн',
 'форм',
 'деятельн',
 'игра',
 'важн',
 'рол',
 'в',
 'формирован',
 'нов',
 'предложен',
 'практическ',
 'оп',
 'показыва',
 'что',
 'нов',
 'модел',
 'организацион',
 'деятельн',
 'игра',
 'важн',
 'рол',
 'в',
 'формирован',
 'направлен',
 'прогрессивн',
 'развит',
 'дорог',
 'друз',
 'нача',
 'повседневн',
 'работ',
 'по',
 'формирован',
 'позиц',
 'позволя',
 'выполн',
 'важн',
 'задан',
 'по',
 'разработк',
 'систем',
 'обучен',
 'кадр',
 'соответств',
 'насущн',
 'потребн',
 'повседневн',
 'практик',
 'показыва',
 'что',
 'р

In [8]:
print('Processing : [=', end='')
df['text'] = df['text'].apply(remove_non_alphabets)
print('=', end='')
df['text'] = df['text'].apply(tokenize) # [ word_tokenize(row) for row in df['text']]
print('=', end='')
df['text'] = df['text'].apply(stem)
print('=', end='')
df['text'] = df['text'].apply(leammtizer)
print('=', end='')
df['text'] = df['text'].apply(lambda x: ' '.join(x))
print('] : Completed', end='')

df.dropna(subset=['text'], inplace=True)
# df.to_csv("sdd.csv")

Processing : [=====] : Completed

In [9]:
df_file = df[['text', 'count_words', 'stars']]
df_file
# df_file.to_csv("file.csv")

,text,count_words,stars
0,книга полна разочаровать автор позиционир себ ...,48,0
1,когд я повернуть не туд поч никта я не сказ чт...,118,0
2,претенциозна ничт как и предыдущий книга в эт ...,22,0
4,врод бы нельз воспринить эт книга буквальна но...,30,0
5,пос у книга важн и нужн отчётливый понятн что ...,126,0
...,...,...,...
1500,книга заслуживый ваш вниман и время настольк о...,119,4
1501,каз бы в книга изложа тот истина котор давна и...,40,4
1502,автор не разочаровыв к мо больша удовольство о...,33,4
1503,красивый и богатый язык тонк юмор классна перс...,126,4


In [14]:
df.dropna(subset=['text'], inplace=True)

abs_path       1447
stars          1447
text           1447
count_words    1447
dtype: int64

In [16]:
df.groupby("stars").count()

for stars, mini_df in df.groupby("stars"):
    print(f"Stars: {stars + 1}")
    print(f"Max: {mini_df.count_words.max()}")
    print(f"Min: {mini_df.count_words.min()}")
    print(f"Mean: {int(mini_df.count_words.mean())}")
    print("===========================================")

Stars: 1
Max: 272
Min: 0
Mean: 101
Stars: 2
Max: 143
Min: 8
Mean: 104
Stars: 3
Max: 146
Min: 3
Mean: 109
Stars: 4
Max: 147
Min: 9
Mean: 110
Stars: 5
Max: 144
Min: 5
Mean: 100


In [114]:
df = df[['text', 'count_words', 'stars']]
df.to_csv(r"file.gz", compression='gzip', index=False)

# AI


In [ ]:

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision
from torch.utils.data import TensorDataset, DataLoader


In [108]:
max_words = 14000
stop_words = set(stopwords.words("russian"))

cv = CountVectorizer(max_features=max_words, stop_words=list(stop_words))
sparse_matrix = cv.fit_transform(df['text']).toarray()
sparse_matrix.shape


(1447, 14000)

In [109]:
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.linear1 = nn.Linear(max_words, 7000)
        self.linear2 = nn.Linear(7000, 3500)
        self.linear3 = nn.Linear(3500, 500)
        self.linear4 = nn.Linear(500, 50)
        self.linear5 = nn.Linear(50, 5)
        
    def forward(self, x):
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = F.relu(self.linear3(x))
        x = F.relu(self.linear4(x))
        x = self.linear5(x)
        return x

In [110]:
x_train, x_temp, y_train, y_temp = train_test_split(sparse_matrix, np.array(df['stars']), train_size=0.8, random_state=42)

x_test, x_val, y_test, y_val = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)

In [111]:
x_train = Variable(torch.from_numpy(x_train)).float()
y_train = Variable(torch.from_numpy(y_train)).long()

x_test = Variable(torch.from_numpy(x_test)).float()
y_test = Variable(torch.from_numpy(y_test)).long()

x_val = Variable(torch.from_numpy(x_val)).float()
y_val = Variable(torch.from_numpy(y_val)).long()

## Обучение модели


In [112]:
index = 0
learning_rates = [0.001, 0.01, 0.1]
batch_sizes = [32, 64, 128]
criterion = nn.CrossEntropyLoss()

models_base = "models_base"
current_path = os.getcwd()

if not os.path.exists(models_base):
    os.mkdir(models_base)

else:
    shutil.rmtree(models_base)
    os.mkdir(models_base)


model_list = []
epochs = 10

for lr in learning_rates:
    for batch_size in batch_sizes:
        # Переопределение оптимизатора с новыми параметрами
        model = SimpleModel()
        optimizer = torch.optim.Adam(params=model.parameters() , lr=lr)
        
        # Инициализация даталоадеров с новым batch_size
        train_loader = torch.utils.data.DataLoader(list(zip(x_train, y_train)), batch_size=batch_size, shuffle=True)
        
        test_loader = torch.utils.data.DataLoader(list(zip(x_test, y_test)), batch_size=batch_size, shuffle=False)
        
        val_loader = torch.utils.data.DataLoader(list(zip(x_val, y_val)), batch_size=batch_size, shuffle=False)
        
        # Обновление переменных
        model.train()
        loss_values_train = []
        acc_values_train = []
        loss_values_val = []
        acc_values_val = []
        loss_values_test = []
        acc_values_test = []

        for epoch in range(epochs):
            model.train()
            running_loss = 0.0
            correct_train = 0

            for inputs, labels in train_loader:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                running_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                correct_train += (predicted == labels).sum().item()

            # Вычисление метрик для обучающей выборки
            average_loss_train = running_loss / len(train_loader)
            accuracy_train = correct_train / len(x_train)
            loss_values_train.append(average_loss_train)
            acc_values_train.append(accuracy_train)

            model_list.append(model)
            
            # Валидация модели
            model.eval()
            running_loss_val = 0.0
            correct_val = 0

            with torch.no_grad():
                for inputs, labels in val_loader:
                    outputs = model(inputs)
                    loss_val = criterion(outputs, labels)
                    running_loss_val += loss_val.item()
                    _, predicted_val = torch.max(outputs.data, 1)
                    correct_val += (predicted_val == labels).sum().item()

            # Вычисление метрик для валидационной выборки
            average_loss_val = running_loss_val / len(val_loader)
            accuracy_val = correct_val / len(x_val)
            loss_values_val.append(average_loss_val)
            acc_values_val.append(accuracy_val)

            
            # Тест модели
            running_loss_test = 0.0
            correct_test = 0

            with torch.no_grad():
                for inputs, labels in test_loader:
                    outputs = model(inputs)
                    loss_test = criterion(outputs, labels)
                    running_loss_test += loss_test.item()
                    _, predicted_test = torch.max(outputs.data, 1)
                    correct_test += (predicted_test == labels).sum().item()

            # Вычисление метрик для тестовой выборки
            average_loss_test = running_loss_test / len(test_loader)
            accuracy_test = correct_test / len(x_test)
            loss_values_test.append(average_loss_test)
            acc_values_test.append(accuracy_test)


            # Вывод результатов каждую эпоху
            print('№{} LR: {}, Batch Size: {}, Epoch: {}/{}, Loss (Train/Val): {:.4f}/{:.4f}, Accuracy (Train/Val): {:.4f}/{:.4f}'.format(
                index, lr, batch_size, epoch + 1, epochs, average_loss_train, average_loss_val, accuracy_train, accuracy_val))
            index += 1
            model_list.append(model)
            # torch.save(model.state_dict(),os.path.join(models_base, f'model_{index}_lr_{lr}_bs_{batch_size}_epoch_{epoch + 1}.pth'))

        # Построение графиков
        plt.figure(figsize=(12, 4))

        # График функции потерь
        plt.subplot(1, 2, 1)
        plt.plot(range(epochs), loss_values_train, label='Train')
        plt.plot(range(epochs), loss_values_val, label='Validation')
        plt.plot(range(epochs), loss_values_test, label='Test')
        plt.title(f'Loss: lr: {lr} bs:{batch_size}')
        plt.xlabel('Epoch')
        plt.ylabel('Value')
        plt.legend()

        # График метрики качества
        plt.subplot(1, 2, 2)
        plt.plot(range(epochs), acc_values_train, label='Train')
        plt.plot(range(epochs), acc_values_val, label='Validation')
        plt.plot(range(epochs), acc_values_test, label='Test')
        plt.title(f'Accuracy: lr:{lr} bs:{batch_size}')
        plt.xlabel('Epoch')
        plt.ylabel('Value')
        plt.legend()

        plt.tight_layout()
        plt.show()

№0 LR: 0.001, Batch Size: 32, Epoch: 1/10, Loss (Train/Val): 1.5609/1.5465, Accuracy (Train/Val): 0.2437/0.2621


KeyboardInterrupt: 

Посмотреть на каких-нибудь данных

## Тренировка моделей закончена, они находятся в model_list


In [104]:
index = 97

In [105]:
new_review = """Как же я долго читала книгу, но мне просто не хочется расставаться с автором, читала книгу по чуть-чуть, наслаждаясь авторской задумкой и неповторимым стилем. Мне горестно от того, что все книги автора уже прочитаны, очень буду ждать новых историй. В этой книге мы более подробно узнаем об мире Ордэне, об их деятельности и их возможностях. Маргарита живет в две тысячи восемнадцатом году, но благодаря своему увлечению куклами и странному продавцу девушка оказывается в прошлом и ей предстоит стать гувернанткой девочки Нади и помочь ей избавиться от странных монстров, а еще разгадать несколько закрученных семейных тайн. Когда в тексте появились часы, где ласточка под другим углом уклона превращается в ворона, у меня сразу возникли подозрения, что Яша — это тот самый Яков из книги Сердце…
"""
new_vectorize = cv.transform([new_review]).toarray()
new_review_tensor = torch.from_numpy(new_vectorize).float()

In [102]:
new_vectorize.shape

(1, 14000)

In [106]:
model_list[index].eval()
with torch.no_grad():
    output = model_list[index](new_review_tensor)
    _, predicted_label = torch.max(output.data, 1)

print(f"Предсказанная метка для нового отзыва: {predicted_label.item() + 1}")

Предсказанная метка для нового отзыва: 1


In [77]:
models_base = "models_base"
current_path = os.getcwd()

if not os.path.exists(models_base):
    os.mkdir(models_base)

else:
    shutil.rmtree(models_base)
    os.mkdir(models_base)

In [ ]:
torch.save(model_list[index].state_dict(), os.path.join(models_base, f'model_{index}.pth'))

## Сохранение готовых моделей
